# Criação do Métricas

## Importação de bibliotecas

In [45]:
#Importar Biblitocas
import pandas as pd
import numpy as np

## Definição de pastas e leitura de arquivos

In [46]:
#Caminho de leitura da base VTEX

#Trampo
#base_vtex_set = r"C:\Users\kevinbotelho\OneDrive - CTC FRANCHISING S A\Área de Trabalho\projetos\bi omnichat\Bases\Base vtex\2025_09_base_vtex.csv"

#Casa
base_vtex_set = r"C:\Users\kevin\OneDrive\Área de Trabalho\projeto\projeto metricas\base vtex\2025_09_base_vtex.csv"

In [ ]:
#Leitura da base
df_vtex = pd.read_csv(
    base_vtex_set,
    sep=';')

In [ ]:
#Verificação dos dados
df_vtex.head()

## Correção das colunas principais e criação da nova tabela

In [49]:
#Transformando os tipos dos dados
df_vtex['Creation Date'] = pd.to_datetime(df_vtex['Creation Date'])
df_vtex['Client Document'] = df_vtex['Client Document'].astype(str)

In [ ]:
#Verificando os tipos dos dados
df_vtex.info()

In [51]:
#Criação da tabela métricas com colunas principais
df_metricas = df_vtex[['Order', 'Creation Date']].copy()

In [52]:
#Renomeado as colunas principais
df_metricas = df_metricas.rename(columns={
    'Order': 'Pedido Completo',
    'Creation Date': 'Data Site'
})

In [ ]:
df_metricas.head()

## Criação das colunas

### Colunas de dados de pedidos

In [ ]:
#Coluna Pedido
df_metricas['Pedido'] = df_metricas['Pedido Completo'].str.split('-').str[0]

#Coluna Split
df_metricas['Split'] = df_metricas['Pedido Completo'].str.split('-').str[1]

df_metricas.head()

### Colunas de dados de cliente

In [55]:
#Coluna Nome Completo
df_metricas['Nome Completo'] = df_vtex['Client Name'].str.strip().str.title() + ' ' + df_vtex['Client Last Name'].str.strip().str.title()

#Coluna Email
df_metricas['Email'] = df_vtex['Email']

#Coluna Telefone
df_metricas['Telefone'] = df_vtex['Phone'].astype(str)

def arrumar_telefone(telefone):
    if len(telefone) == 11:
        return '55' + telefone
    else:
        return telefone

df_metricas['Telefone'] = df_metricas['Telefone'].apply(arrumar_telefone)

#Coluna CPF
df_metricas['CPF'] = df_vtex['Client Document'].str.zfill(11)

### Colunas de dados de atribuição de vendas

In [56]:
#Construir

### Colunas de dados de produtos

In [57]:
#Coluna Produto
df_metricas['Produto'] = df_vtex['SKU Name'].str.rsplit(' ',n=1).str[0].str.strip()

#Coluna SubCategoria
dict_depara_subcategoria = {
    'Porta-Cartão' : 'Porta-Cartões',
    'Sandálias' : 'Sandália',
    'Tamanho' : 'Tamanco',
    'Tenis' : 'Tênis'
}

df_metricas['Subcategoria'] = df_vtex['SKU Name'].str.split(' ',n=1).str[0].str.strip().str.title()
df_metricas['Subcategoria'] = df_metricas['Subcategoria'].replace(dict_depara_subcategoria)

#Coluna Categoria
lista_sapatos = ['Rasteira', 'Sandália', 'Sapatilha', 'Chinelo', 'Tamanco', 'Mule', 'Papete', 'Scarpin', 'Anabela', 'Coturno', 'Ankle', 'Tênis', 'Slip', 'Bota', 'Mocassim', 'Alpargata']
lista_bolsas = ['Bolsa', 'Carteira', 'Mochila', 'Pasta', 'Mini', 'Porta-Cartão', 'Porta-Cartões', 'Nécessaire', 'Clutch']
lista_acessorios = ['Porta', 'Meia', 'Calcanheira', 'Palmilha', 'Ajuste', 'Protetor']

def definir_categoria(subcategoria):
    if subcategoria in lista_sapatos:
        return 'Sapato'
    elif subcategoria in lista_bolsas:
        return 'Bolsa'
    elif subcategoria in lista_acessorios:
        return 'Acessório'
    else:
        return 'Não Definido'

df_metricas['Categoria'] = df_metricas['Subcategoria'].apply(definir_categoria)

#Coluna SKU
df_metricas['SKU'] = df_vtex['Reference Code'].str.split('_').str[0].str.strip()

#Coluna Tamanho
df_metricas['Tamanho'] = df_vtex['SKU Name'].str.rsplit(' ',n=1).str[1].str.strip()

### Colunas de dados financeiros

In [58]:
#Coluna Preço
df_metricas['Preço Un.'] = df_vtex['SKU Value']

#Coluna Quantidade de Itens
df_metricas['Qtde. Itens'] = df_vtex['Quantity_SKU']

#Coluna Frete
df_metricas['Frete'] = df_vtex['Shipping Value']

#Coluna Desconto
df_metricas['Desconto'] = ((df_vtex['SKU Value'] - df_vtex['SKU Selling Price']) * df_vtex['Quantity_SKU']) * -1

#Coluna Total Pedido
df_metricas['Total Un. C/F'] = (df_metricas['Preço Un.'] * df_metricas['Qtde. Itens']) + df_metricas['Frete'] + df_metricas['Desconto']



### Colunas de dados de frete

In [59]:
def definir_frete(tipo_frete):
    if tipo_frete == 'residential':
        return 'Entrega'
    elif tipo_frete == 'pickup':
        return 'Retirada'
    else:
        return 'Não Definido'

df_metricas['Frete E/R'] = df_vtex['Address Type'].apply(definir_frete)

### Colunas de dados de entrega

In [60]:
#Coluna Código
df_metricas['Código'] = df_vtex['Seller Name']

df_metricas['Código'] = df_metricas['Código'].replace('CONSTANCE','900').str.replace('constance','')

df_metricas['Código'] = df_metricas['Código'].astype(int)

df_metricas['Código'] = df_metricas['Código'].astype(str)

#Coluna Modelo
lista_cd = ['900', '903', '904']
lista_outlet = ['76', '880', '881', '884', '885', '886', '887']

def definir_modelo(modelo):
    if modelo in lista_cd:
        return 'CD'
    elif modelo in lista_outlet:
        return 'Outlet'
    else:
        return 'Loja'
    
df_metricas['Modelo'] = df_metricas['Código'].apply(definir_modelo)

### Colunas de dados de recebimento

In [61]:
#Coluna Cidade
df_metricas['Cidade'] = df_vtex['City']

#Coluna Estado
df_metricas['UF'] = df_vtex['UF']

#Coluna Região
lista_sudeste = ['ES', 'MG', 'RJ', 'SP']
lista_centro_oeste = ['DF', 'GO', 'MS', 'MT']
lista_sul = ['PR', 'RS', 'SC']
lista_nordeste = ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']
lista_norte = ['AM', 'AP', 'PA', 'RO', 'RR', 'TO']

def definir_regiao(uf):
    if uf in lista_sudeste:
        return 'Sudeste'
    elif uf in lista_centro_oeste:
        return 'Centro-Oeste'
    elif uf in lista_sul:
        return 'Sul'
    elif uf in lista_nordeste:
        return 'Nordeste'
    elif uf in lista_norte:
        return 'Norte'
    else:
        return 'Não Identificado'

df_metricas['Região'] = df_metricas['UF'].apply(definir_regiao)

### Colunas de dados de cancelamento

#### Lendo tabela do use

In [62]:
dados_use = r"C:\Users\kevin\OneDrive\Área de Trabalho\projeto\projeto metricas\base use\Relatório Ecommerce por itens - 2025-11-06.xls"

tabelas_use = pd.read_html(dados_use)

df_use = tabelas_use[2][['Pedido Site', 'Referencia', 'Tamanho', 'Status Pedido']].copy()

del tabelas_use

df_use.dropna(inplace=True)

df_use = df_use[df_use['Status Pedido'] == 'CANCELADO']

df_use['Tamanho'] = df_use['Tamanho'].astype(int).astype(str)

df_use['ID Canc Use'] = df_use['Pedido Site'] + '_' + df_use['Referencia'] + '_' + df_use['Tamanho']

ids_cancelados_use = set(df_use['ID Canc Use'])

#### Lendo tabela do shankya

In [63]:
dados_shankya = r"C:\Users\kevin\OneDrive\Área de Trabalho\projeto\projeto metricas\base sankhya\arquivo.xlsx"

df_sankhya = pd.read_excel(dados_shankya, skiprows= 2)[['Nro. Pedido', 'Referência', 'Controle', 'Quantidade Corte']]

df_sankhya = df_sankhya[df_sankhya['Quantidade Corte'] == 1].reset_index(drop=True)

df_sankhya['Controle'] = df_sankhya['Controle'].astype(int).astype(str)

df_sankhya['ID Canc Shankya'] = df_sankhya['Nro. Pedido'] + '_' + df_sankhya['Referência'] + '_' + df_sankhya['Controle']

ids_cancelados_sankhya = set(df_sankhya['ID Canc Shankya'])

#### Criação das colunas

In [ ]:
#Coluna ID Cancelamento
df_metricas['ID Canc Vtex'] = df_metricas['Pedido Completo'] + '_' + df_metricas['SKU'] + '_' + df_metricas['Tamanho']

#Coluna Status
df_metricas['Status'] = df_vtex['Status']

cond_1 = (df_metricas['Status'] == 'Cancelado') 
cond_2 = df_metricas['ID Canc Vtex'].isin(ids_cancelados_sankhya)
cond_3 = df_metricas['ID Canc Vtex'].isin(ids_cancelados_use)

condicao_final = (cond_1 | cond_2 | cond_3)
df_metricas['Cancelado'] = np.where(condicao_final, 'Sim', 'Não')

#Deletando Colunas de Status de ID
df_metricas = df_metricas.drop(columns=['ID Canc Vtex', 'Status'])

### Colunas de dados de promoção

In [65]:
#Coluna Promoção
df_metricas['Promoção'] = 0

#Coluna Cupom
df_metricas['Cupom'] = df_vtex['Coupon'].apply(lambda x: 'Não' if pd.isna(x) else 'Sim')

#Coluna Nome Cupom
df_metricas['Nome Cupom'] = df_vtex['Coupon'].str.upper()
df_metricas['Nome Cupom'] = df_metricas['Nome Cupom'].fillna('Sem Cupom')

#Coluna Campanha
df_metricas['Campanha'] = df_vtex['Discounts Names']
df_metricas['Campanha'] = df_metricas['Campanha'].fillna('Sem Campanha')

### Teste DF

In [77]:
df_metricas.head()

,Pedido Completo,Data Site,Pedido,Split,Nome Completo,Email,Telefone,CPF,Produto,Subcategoria,...,Código,Modelo,Cidade,UF,Região,Cancelado,Promoção,Cupom,Nome Cupom,Campanha
0,1565542266853-02,2025-09-30 23:17:53,1565542266853,02,Ana Giulia Bianchi,anagiuliabianchigalvao@outlook.com,5516999861601,18189155881,Rasteira de Tiras Enfeites Preta,Rasteira,...,516,Loja,Miguelópolis,SP,Sudeste,Não,0,Não,Sem Cupom,Sem Campanha
1,1565552266890-01,2025-09-30 23:59:23,1565552266890,01,Marleide Josefa Da Silva,marl.silva@hotmail.com,5515988077447,27506852810,Sandália Fisherman Tratorada,Sandália,...,900,CD,Sorocaba,SP,Sudeste,Não,0,Não,Sem Cupom,Sem Campanha
2,1565542266767-01,2025-09-30 22:30:29,1565542266767,01,Natalia Jeniffer Souza Rodrigues,nat.jheniffer@gmail.com,5592984749035,70084201290,Sandália Preta Flores Strass,Sandália,...,472,Loja,Manaus,AM,Norte,Não,0,Sim,CINDERELA10,Desconto Influencers Jan/Dez 2025
3,1565542266767-01,2025-09-30 22:30:29,1565542266767,01,Natalia Jeniffer Souza Rodrigues,nat.jheniffer@gmail.com,5592984749035,70084201290,Rasteira Marrom Tiras Strass,Rasteira,...,472,Loja,Manaus,AM,Norte,Não,0,Sim,CINDERELA10,Desconto Influencers Jan/Dez 2025
4,1565542266767-01,2025-09-30 22:30:29,1565542266767,01,Natalia Jeniffer Souza Rodrigues,nat.jheniffer@gmail.com,5592984749035,70084201290,Sandália Off White Elo,Sandália,...,472,Loja,Manaus,AM,Norte,Não,0,Sim,CINDERELA10,Desconto Influencers Jan/Dez 2025


In [75]:
df_metricas['Data Site'] = df_metricas['Data Site'].dt.tz_localize(None)
df_metricas[df_metricas['Cancelado'] == 'Sim'].to_excel(
    r'C:\Users\kevin\OneDrive\Área de Trabalho\projeto\projeto metricas\validacao.xlsx',
    index=False
)

In [ ]:
df_metricas.info()